In [1]:
import os, sys
os.environ['HADOOP_CONF_DIR'] = '/etc/hadoop/conf'
os.environ['YARN_CONF_DIR'] = '/etc/hadoop/conf'
os.environ['PYSPARK_PYTHON'] =  'python3.9'
os.environ['PYSPARK_DRIVER_PYTHON'] = 'python3.9'
os.environ['HADOOP_USER_NAME']='ssenigov'

from pyspark.sql import SparkSession

In [ ]:
spark = (SparkSession.builder.appName("window_lead_lag_nulls")
         .master("yarn").getOrCreate())

print('app_id'.ljust(40), spark.sparkContext.applicationId)

In [9]:
sql = """
   select * from values 
    (0, null), 
    (1, 'X'), 
    (2, null),
    (3, null),
    (4, 'Y'),
    (5, null) as (id, val)
"""
df = spark.sql(sql)
df.createOrReplaceTempView('tbl_with_nulls')
spark.table('tbl_with_nulls').show()

+---+----+
| id| val|
+---+----+
|  0|null|
|  1|   X|
|  2|null|
|  3|null|
|  4|   Y|
|  5|null|
+---+----+



In [44]:
sql = """
  select 
   val,
    LEAD(val, 1, 0) IGNORE NULLS over w lead_ignore_nulls,
    LEAD(val, 1, 0) over w lead,
    '' as ___,
   val,
    LAG(val, 1, 0)  IGNORE NULLS over w lag_ignore_nulls,
    LAG(val, 1, 0)  over w lag
  from tbl_with_nulls 
   window w as (partition by '1' order by id)
  order by id
"""
df = spark.sql(sql)
df.show()

+----+-----------------+----+---+----+----------------+----+
| val|lead_ignore_nulls|lead|___| val|lag_ignore_nulls| lag|
+----+-----------------+----+---+----+----------------+----+
|null|                X|   X|   |null|               0|   0|
|   X|                Y|null|   |   X|               0|null|
|null|                Y|null|   |null|               X|   X|
|null|                Y|   Y|   |null|               X|null|
|   Y|                0|null|   |   Y|               X|null|
|null|                0|   0|   |null|               Y|   Y|
+----+-----------------+----+---+----+----------------+----+



In [ ]:
spark.stop()